<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/383_GCO_ReportGen_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Report generation utilities for Governance & Compliance Orchestrator

Generates comprehensive audit reports in markdown format.
"""

from typing import Dict, Any, List, Optional
from datetime import datetime
from pathlib import Path
import os


def generate_audit_report(state: Dict[str, Any]) -> str:
    """
    Generate comprehensive audit report in markdown format.

    Args:
        state: Complete orchestrator state

    Returns:
        Markdown report string
    """
    summary = state.get("summary", {})
    prioritized_issues = state.get("prioritized_issues", [])
    risk_scores = state.get("risk_scores", {})
    bias_signals = state.get("bias_signals", [])
    drift_signals = state.get("drift_signals", [])

    report = f"""# Governance & Compliance Audit Report

**Generated:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

---

## Executive Summary

- **Total Events Analyzed:** {summary.get('total_events_analyzed', 0)}
- **Total Violations Detected:** {summary.get('total_violations', 0)}
- **High/Critical Severity Issues:** {summary.get('high_severity_count', 0)}
- **Bias Signals:** {summary.get('bias_signals_count', 0)}
- **Drift Signals:** {summary.get('drift_signals_count', 0)}
- **Overall Risk Score:** {risk_scores.get('overall_risk_score', 0.0):.2f}/1.0

### Risk Level Assessment

"""

    overall_risk = risk_scores.get("overall_risk_score", 0.0)
    if overall_risk >= 0.75:
        risk_level = "🔴 **CRITICAL** - Immediate action required"
    elif overall_risk >= 0.50:
        risk_level = "🟠 **HIGH** - Urgent review needed"
    elif overall_risk >= 0.25:
        risk_level = "🟡 **MEDIUM** - Review recommended"
    else:
        risk_level = "🟢 **LOW** - Monitor and maintain"

    report += f"{risk_level}\n\n"

    # Severity Breakdown
    severity_breakdown = summary.get("severity_breakdown", {})
    report += f"""### Severity Breakdown

- **Critical:** {severity_breakdown.get('critical', 0)}
- **High:** {severity_breakdown.get('high', 0)}
- **Medium:** {severity_breakdown.get('medium', 0)}
- **Low:** {severity_breakdown.get('low', 0)}

---

## Top Priority Issues

"""

    # Top 10 prioritized issues
    for i, issue in enumerate(prioritized_issues[:10], 1):
        report += f"""### {i}. {issue.get('policy_id', 'Unknown Policy')} - {issue.get('severity', 'medium').upper()} Severity

- **Event ID:** {issue.get('event_id', 'N/A')}
- **Priority Score:** {issue.get('priority_score', 0.0):.1f}/100
- **Risk Type:** {issue.get('risk_type', 'policy_violation')}
- **Recommended Action:** {issue.get('recommended_action', 'Review')}
- **Reason:** {issue.get('reason', 'No reason provided')}

"""

    # Agent Risk Scores
    agent_scores = risk_scores.get("agent_scores", {})
    if agent_scores:
        report += "---\n\n## Agent Risk Scores\n\n"
        for agent_name, score_data in sorted(
            agent_scores.items(),
            key=lambda x: x[1].get("risk_score", 0.0),
            reverse=True
        ):
            report += f"""### {agent_name}

- **Risk Score:** {score_data.get('risk_score', 0.0):.2f}/1.0
- **Total Violations:** {score_data.get('total_violations', 0)}
- **Bias Signals:** {score_data.get('bias_signals_count', 0)}
- **Drift Signals:** {score_data.get('drift_signals_count', 0)}

"""

    # Bias Signals
    if bias_signals:
        report += "---\n\n## Bias Detection Signals\n\n"
        for signal in bias_signals[:5]:  # Top 5
            report += f"""### {signal.get('signal_id', 'Unknown')} - {signal.get('risk_level', 'medium').upper()} Risk

- **Agent:** {signal.get('agent_name', 'Unknown')}
- **Decision Type:** {signal.get('decision_type', 'Unknown')}
- **Protected Attribute:** {signal.get('protected_attribute', 'Unknown')}
- **Delta:** {signal.get('delta', 0.0):.2f} (Threshold: {signal.get('threshold', 0.0):.2f})
- **Recommended Action:** {signal.get('recommended_action', 'Review')}

"""

    # Drift Signals
    if drift_signals:
        report += "---\n\n## Drift & Degradation Signals\n\n"
        for signal in drift_signals[:5]:  # Top 5
            report += f"""### {signal.get('signal_id', 'Unknown')} - {signal.get('risk_level', 'medium').upper()} Risk

- **Agent:** {signal.get('agent_name', 'Unknown')}
- **Metric:** {signal.get('metric', 'Unknown')}
- **Previous Average:** {signal.get('previous_average', 0.0):.3f}
- **Current Average:** {signal.get('current_average', 0.0):.3f}
- **Delta:** {signal.get('delta', 0.0):.3f} (Threshold: {signal.get('threshold', 0.0):.3f})
- **Recommended Action:** {signal.get('recommended_action', 'Review')}

"""

    report += """---

## Recommendations

1. **Immediate Actions:**
   - Address all critical and high severity violations
   - Escalate blocking issues to compliance officers
   - Review and update policies as needed

2. **Short-term Actions:**
   - Implement human-in-the-loop requirements for high-risk decisions
   - Review bias signals and adjust model training if needed
   - Monitor drift signals and consider model retraining

3. **Long-term Actions:**
   - Establish continuous monitoring processes
   - Regular compliance audits
   - Policy refinement based on findings

---

*This report was generated by the Governance & Compliance Orchestrator Agent.*
"""

    return report


def save_report(
    report_content: str,
    reports_dir: str,
    timestamp: Optional[str] = None
) -> str:
    """
    Save report to file.

    Args:
        report_content: Report content (markdown)
        reports_dir: Directory to save reports
        timestamp: Optional timestamp string (defaults to current time)

    Returns:
        Path to saved report file
    """
    os.makedirs(reports_dir, exist_ok=True)

    if timestamp is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    filename = f"governance_audit_{timestamp}.md"
    filepath = os.path.join(reports_dir, filename)

    with open(filepath, 'w') as f:
        f.write(report_content)

    return filepath

